In [1]:
import fcalc

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score

Reading data

In [4]:
all_data = pd.read_csv('Mushrooms.csv')
all_data.loc[all_data['class'] == 'p', 'class'] = 0
all_data.loc[all_data['class'] == 'e', 'class'] = 1
all_data['class'] = all_data['class'].astype(int)

all_data, _ = train_test_split(all_data, train_size=0.3, stratify=all_data['class'])

X, y = all_data.drop(columns = 'class'), all_data['class']

Binarizing data

In [14]:
X = pd.get_dummies(X)

In [15]:
X.shape

(2437, 117)

Assesing performance

In [16]:
def cross_val_score(X, y, n_splits, scoring):
    skf = StratifiedKFold(n_splits=n_splits, random_state=42, shuffle=True)
    scores = np.zeros((len(scoring), skf.get_n_splits()))

    for i, (train_index, test_index) in enumerate(skf.split(X, y)):
        X_train, y_train = X.iloc[train_index], y.iloc[train_index]
        X_test, y_test = X.iloc[test_index], y.iloc[test_index]

        bin_cls = fcalc.classifier.BinarizedBinaryClassifier(X_train.values, y_train.values)
        bin_cls.predict(X_test.values)

        # we always classify non prediction as false prediction
        tmp_predict = np.copy(bin_cls.predictions)
        tmp_predict[tmp_predict == -1] = np.abs(y_test.values[tmp_predict == -1] - 1)

        for i, score in enumerate(scoring):
            scores[i] = score(y_test.values, tmp_predict)
    
    return scores.mean(axis = 1).round(3)


In [ ]:
scores = cross_val_score(X, y, 5, [accuracy_score, f1_score])

In [12]:
scores

array([0.98765432, 0.98809524])